In [1]:
#import relevant libraries
import os
import pandas as pd 
import numpy as np

**Step 1 load data**

In [2]:
#load data
aisles = pd.read_csv('data/datasets_4931_7487_aisles.csv') 
departments = pd.read_csv('data/datasets_4931_7487_departments.csv')
products_prior = pd.read_csv('data/order_products__prior.csv')
products_train = pd.read_csv('data/order_products__train.csv')
orders = pd.read_csv('data/orders.csv')
products = pd.read_csv('data/products.csv')

* Aisles: This set includes all the aisles, defined by aisle ID and aisle name 
* Departments: This set includes all the departments, defined by department ID and department name
* the Products Prior: Includes all the products in each order id prior to their final order, defined by order ID, product ID, add to cart order number, and reordered column (target variable) 
* Products Train: Includes all the products in the final order for each customer within the dataset, defined by order ID, product ID, add to cart order number, and reordered column (target variable). 
* Orders: This table includes all orders in the train, test, and prior sets that were split by Instacart and defined by order ID, user ID, order number, order day of the week, order hour of the day, and days since prior order 
* Products: This table includes all products Instacart offers, defined by product ID, product name, aisle ID, department ID 


Aisles df key id : aisle_id

In [3]:
#inspect data 
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


departments key id: department_id 

In [4]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


products key id: order_id, product_id 
gives order added to cart, and if reordered or not 

In [5]:
products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


products key id: order_id, product_id 
gives order added to cart, and if reordered or not 

In [6]:
products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


key id: order id, eval_set : gives which set the order is from, order_dow: day of week

In [7]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


key id : product id 

In [8]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


**Step 2 merge dataframes**

Create a single data frame from product, aisles, and department using aisleID and departmentID as keys. We will use an inner merge since there should be an aisle and department for each product. 



In [9]:

product_info = products.merge(aisles, on = 'aisle_id',  how='inner')
product_info.head()

,product_id,product_name,aisle_id,department_id,aisle
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes
2,102,Danish Butter Cookies,61,19,cookies cakes
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes


In [10]:
product_info = product_info.merge(departments, on = 'department_id', how='inner')
product_info.head()

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes,snacks
2,102,Danish Butter Cookies,61,19,cookies cakes,snacks
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes,snacks
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes,snacks


In [11]:
#merge orders and products_prior : 

prior_orders = orders.merge(products_prior, on = 'order_id', how = 'inner')
prior_orders = prior_orders.merge(product_info, on = 'product_id', how = 'inner')
prior_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,soft drinks,beverages
1,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7,soft drinks,beverages
2,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7,soft drinks,beverages
3,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7,soft drinks,beverages
4,431534,1,prior,5,4,15,28.0,196,1,1,Soda,77,7,soft drinks,beverages


In [12]:
# merge orders and products train:

train_orders = orders.merge(products_train, on = 'order_id', how = 'inner')
train_orders = train_orders.merge(product_info, on = 'product_id', how = 'inner')
train_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,1187899,1,train,11,4,8,14.0,196,1,1,Soda,77,7,soft drinks,beverages
1,2757217,67,train,25,0,11,5.0,196,1,1,Soda,77,7,soft drinks,beverages
2,632715,676,train,12,0,13,26.0,196,2,1,Soda,77,7,soft drinks,beverages
3,1167274,760,train,5,4,10,8.0,196,2,1,Soda,77,7,soft drinks,beverages
4,3347074,804,train,16,3,21,5.0,196,4,0,Soda,77,7,soft drinks,beverages


In [13]:
# Create master dataframe with both train and prior orders

df  = pd.concat((prior_orders, train_orders))


**Step 3 Data Definition**

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33819106 entries, 0 to 1384616
Data columns (total 15 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
 7   product_id              int64  
 8   add_to_cart_order       int64  
 9   reordered               int64  
 10  product_name            object 
 11  aisle_id                int64  
 12  department_id           int64  
 13  aisle                   object 
 14  department              object 
dtypes: float64(1), int64(10), object(4)
memory usage: 4.0+ GB


In [15]:
#change type to category
df['aisle'] = df['aisle'].astype('category')
df['department'] = df['department'].astype('category')
df['eval_set'] = df['eval_set'].astype('category')
df['product_name'] = df['product_name'].astype('category')


**Step 4 : Data Cleaning**

In [16]:
df.isnull().values.sum() 

2078068

In [17]:
nas=pd.DataFrame(df.isnull().sum().sort_values(ascending=False)/len(prior_orders),columns = ['percent'])
pos = nas['percent'] > 0
nas[pos]

,percent
days_since_prior_order,0.06407


Since these null values cannot be replaced with zero or dropped because they are the first order of each customer, we will keep them. 

In [ ]:
#export to csv to use for EDA and preprocessing

df.to_csv('data/merged.csv')